In [1]:
import pandas as pd
import numpy as np
import scipy.special
from scipy.stats import chi2

In [2]:
data = pd.DataFrame(
    {
        "st zlomov": pd.Series(range(6)),
        "st palic": pd.Series([157, 69, 35, 17, 1, 1])
    }
)

In [3]:
data


,st zlomov,st palic
0,0,157
1,1,69
2,2,35
3,3,17
4,4,1
5,5,1


In [4]:
n = sum(data['st palic'])
X_bar = sum(data['st palic']*data['st zlomov']) / n

p_hat = X_bar/5

In [5]:
def bin_distrib(x, p, m=5):
    return scipy.special.comb(m, x, exact=True) * p**x * (1 - p)**(m - x)

In [31]:
# za Pearsonov hi kvadrat test zahtevamo, da je minimum po celici vsaj 5,
# zato smo celici 4 in 5 pridružili celici 3.
O = pd.Series(data[data['st palic'] > 5]['st palic'])
O[3] += data['st palic'][4] + data['st palic'][5]

In [32]:
# izračunajmo pričakovane rezultate po celicah, kjer upoštevamo, 
# da smo celici 3 pridružili celici 4 in 5, torej je potrebno prilagoditi
#  E_3 = n * (p_3 + p_4 + p_5) 

E = pd.Series([n * bin_distrib(x, p_hat, 5) for x in range(4)])
E[3] += n * (bin_distrib(4, p_hat, 5) + bin_distrib(5, p_hat, 5))

X2 = sum((O - E)**2 / E)
print(X2)

44.148661018431994


In [33]:
alpha_1 = 0.05
alpha_2 = 0.01

# določimo še prostostne stopnje chi^2 porazdelitve 
# (po novem) imamo 4 celice, en parameter p_hat smo ocenili iz podatkov, 
# zato bomo imeli 2 prostostni stopnji.
print(chi2.ppf(1 - alpha_1, 2))
print(chi2.ppf(1 - alpha_2, 2))

# testna statistika X2 je na našem vzorcu večja, 
# zato ničelno hipotezo zavržemo tako pri stopnji tveganja 5% kot tudi 1%.

5.991464547107979
9.21034037197618


In [34]:
O

0    157
1     69
2     35
3     19
Name: st palic, dtype: int64

In [35]:
E

0    130.086698
1    107.773742
2     35.715195
3      6.424365
dtype: float64

In [91]:
X = []
for k in range(6):
    X += [k] * data['st palic'][k]

Lambda = np.prod([
    ( (X_bar**x) * ((5 - X_bar)**(5 - x)) ) / ( (x**x) * ((5 - x)**(5 - x)) )
    for x in X]
    )

def aux(x, x_bar):
    if x == 0:
        return (5 - x) * np.log((5 - x_bar) / (5 - x))
    elif x == 5:
        return x * np.log(x_bar/x)
    else:
        return x * np.log(x_bar/x) + (5 - x) * np.log((5 - x_bar) / (5 - x))

logLambda = sum(aux(x, X_bar) for x in X)

# logLambda = (
#     sum(x * np.log(X_bar/x) for x in X if x != 0) +
#     sum((5 - x) * np.log((5 - X_bar) / (5 - x)) for x in X if x != 5)
# )

# zaradi boljše stabilnosti numeričnega računanja bomo rajši delali s statistiko logLambda
# računati x^x in množiti je verjetno manj stabilno kot logaritmiranje in seštevanje 
print(np.log(Lambda))
print(logLambda)

-222.24428374568322
-222.24428374568282


In [44]:
Lambda = np.prod([((X_bar**x)/(x**x) * (5 - X_bar)**(5 - x)/(5 - x)**(5 - x))**data['st palic'][x] for x in range(6)])

np.log(Lambda)

-222.24428374568322

In [93]:
# ta funkcija bo ustvarila vrednost Bin(m, p) porazdeljene slučajne spremenljivke
def binomial_random_number(m=5, p=p_hat): 
    return np.random.choice(np.arange(0,6), p=[bin_distrib(x,p,m) for x in range(6)])

def generate_sample(n):
    X = []
    for _ in range(n):
        X.append(binomial_random_number(5, p_hat))
    return X

count = 0
for _ in range(1000):
    X = generate_sample(n)
    x_bar = sum(X)/len(X)
    
    logL = sum(aux(x, x_bar) for x in X)
    # logL = (
    #     sum(x * np.log(X_bar/x) for x in X if x != 0) +
    #     sum((5 - x) * np.log((5 - X_bar) / (5 - x)) for x in X if x != 5)
    # )
    # print(logL)
    # print(x_bar)
    
    if logL > logLambda:
        count += 1

count

-162.21022336980994
-160.02394816914148
-162.41350596866056
-156.0745757744738
-164.19930330637354
-175.0858409501868
-157.86704875860906
-136.9045873005939
-161.17402269031214
-162.71486241805346
-153.27166191714133
-152.53256087853958
-166.48078895532083
-171.0999861616991
-159.85245305074602
-171.9875594235272
-168.5979740440082
-156.41929751939554
-163.16871485651123
-142.38583710615168
-151.50635541563258
-158.93622575458926
-170.8264550946535
-171.83467341700563
-167.14457599118853
-167.8560686232142
-166.46735745231192
-144.11192685211483
-172.73391052703641
-175.05066993775816
-167.50781494030682
-151.32604539197925
-155.3619239501201
-160.36594818550205
-167.306623188341
-153.66144327394235
-159.82436731659456
-154.06892458866722
-153.53083897614218
-159.803656521465
-138.15680456032072
-167.72988629165445
-165.4090516509782
-166.768713901705
-141.9953232654502
-169.3718694577029
-162.69787893034126
-161.83591791567127
-144.50976880047747
-160.0239481691415
-156.4609466375265


1000